# Learning Python as an R user

# Import libraries

In [383]:
import pandas
import math
import numpy
import plotly.express
import scipy
import statsmodels.api
import statsmodels.formula.api
import patsy.contrasts

# Generate data
## `group` variable with 4 levels

In [384]:
group = numpy.repeat(["A", "B", "C", "D"], repeats = 25)

## Save parameters

In [385]:
mean = (3, 4, 4, 3)
sigma = (1, 1, 1, 1)
n = (25, 25, 25, 25)

## Random normal variable `x` whose means depend on levels of `group` variable

In [386]:
# Empty list for appending results
x = []

# Loop through equal-length mean, sigma, and n (group size)
for (i, j, k) in zip(mean, sigma, n):
    x.append(numpy.random.normal(loc = i, scale = j, size = k))

# Concatenate transforms the result into 1 array
x = numpy.concatenate(x)

## Variable `y` correlated r = 0.75 with x

In [387]:
y = x * 0.75 + numpy.random.normal(loc = 0, scale = 1, size = sum(n))

## Store variables in a data frame

In [388]:
data1 = pandas.DataFrame({"y": y, "x": x, "group": group})

## Create contrast variables for use in linear regression on `group` variable

In [389]:
# Helmert contrasts
group_helmert = patsy.contrasts.Helmert().code_without_intercept(list(set(group)))

# 2 main effects and 1 interaction contrast
group_factorial = patsy.contrasts.ContrastMatrix([[-1, -1, 1], [-1, 1, -1], [1, -1, -1], [1, 1, 1]], 
                                                 ["Main Effect 1", "Main Effect 2", "Interaction"])

## Add contrast variables to data frame

In [481]:
def recode(x, x_levels, new_levels):
    for i, xi in enumerate(x):
        for j in range(len(x_levels)):
            if xi == x_levels[j]:
                x[i] = new_levels[j]
        return x
    
data1.loc[data1["group"] == "A", "main1"] = -1
data1.loc[data1["group"] == "B", "main1"] = -1
data1.loc[data1["group"] == "C", "main1"] = 1
data1.loc[data1["group"] == "D", "main1"] = 1

data1.loc[data1["group"] == "A", "main2"] = -1
data1.loc[data1["group"] == "B", "main2"] = 1
data1.loc[data1["group"] == "C", "main2"] = -1
data1.loc[data1["group"] == "D", "main2"] = 1

data1.loc[data1["group"] == "A", "interaction"] = 1
data1.loc[data1["group"] == "B", "interaction"] = -1
data1.loc[data1["group"] == "C", "interaction"] = -1
data1.loc[data1["group"] == "D", "interaction"] = 1

xx = [1, 2, 3, 1, 1, 1, 2, 2, 2, 3, 3, 3]
xlevels = [1, 2, 3]
newlevels = ["A", "B", "C"]
recode(xx, xlevels, newlevels)

['A', 2, 3, 1, 1, 1, 2, 2, 2, 3, 3, 3]

# Plots
## Boxplots

In [333]:
# boxplots of y by group
plotly.express.box(data1, x = "group", y = "y")

In [334]:
# boxplots of x by group
plotly.express.box(data1, x = "group", y = "x")

## Histograms

In [335]:
# y histograms by group
plotly.express.histogram(data1, x = "y", facet_col = "group")

In [336]:
# x histograms by group
plotly.express.histogram(data1, x = "x", facet_col = "group")

## Scatterplot

In [337]:
plotly.express.scatter(data1, x = "x", y = "y", color = "group", trendline = "ols", facet_col = "group")

## Bars of `group` means with 95% confidence intervals

In [378]:
group_desc = data1.groupby("group")["y"].agg(["mean", "sem", "count"]).reset_index()
group_desc["df"] = group_desc["count"] - 1
group_desc["lower"] = group_desc["mean"] - scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]
group_desc["upper"] = group_desc["mean"] + scipy.stats.t.ppf(1 - 0.05 / 2, df = group_desc["df"]) * group_desc["sem"]

# Plot
plotly.express.bar(group_desc, x = "group", y = "mean", error_y_minus = "lower", error_y = "upper", color = "group")

# Descriptive Statistics

In [338]:
data1.groupby("group")[["x", "y"]].describe().round(2)

x                                               y                    \
      count  mean   std   min   25%   50%   75%   max count  mean   std   min   
group                                                                           
A      25.0  2.87  0.83  1.11  2.39  2.85  3.43  4.55  25.0  1.91  1.23  0.06   
B      25.0  4.03  1.09  1.69  3.26  3.80  4.64  6.17  25.0  3.00  1.27  0.40   
C      25.0  3.56  0.94  0.99  3.09  3.70  4.23  5.10  25.0  2.43  0.91  0.90   
D      25.0  2.87  1.05  0.52  2.37  2.71  3.23  5.53  25.0  2.29  1.17  0.21   

                               
        25%   50%   75%   max  
group                          
A      0.94  1.67  2.84  4.18  
B      2.04  2.81  3.93  6.33  
C      1.76  2.40  2.89  4.55  
D      1.52  2.32  2.89  5.09

# Correlation

In [339]:
# Save r and p-value
r1, pvalue1 = scipy.stats.pearsonr(x, y)

# Save degrees of freedom
ddf1 = len(x) - 2

# Compute t-statistic and degress of freedom, isf for upper tail of t distribution
t1 = scipy.stats.t.isf(pvalue1 / 2, df = ddf1)

# Compute standard error
se1 = r1 / t1

# Compute lower and upper confidence intervals
lower1, upper1 = (numpy.tanh(numpy.arctanh(r1) - 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)),
                  numpy.tanh(numpy.arctanh(r1) + 1 / math.sqrt(len(x) - 3) * scipy.stats.norm.ppf(1 - 0.05 / 2)))

# Print most basic results
"r = {0}, 95%CI [{1}, {2}], t({3}) = {4}, p = {5}".format(r1.round(2), lower1.round(2), upper1.round(2), ddf1, t1.round(2), pvalue1.round(3))

'r = 0.62, 95%CI [0.49, 0.73], t(98) = 7.87, p = 0.0'

# Analyses
## Regression
### Fit linear regression

In [340]:
# regress y on numeric/continuous x
ols_fit1 = statsmodels.formula.api.ols("y ~ x", data = data1).fit()

### Results Summary

In [341]:
ols_fit1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     61.96
Date:                Sat, 11 Jan 2020   Prob (F-statistic):           4.77e-12
Time:                        13:20:21   Log-Likelihood:                -135.39
No. Observations:                 100   AIC:                             274.8
Df Residuals:                      98   BIC:                             280.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1157      0.306      0.378      0.707      -0.492       0.724
x              0.6885      0.087      7.871      0.000       0.515       0.862
==============================================================================
Omnibus:                        2.141   Durbin-Watson:                   1.805
Prob(Omnibus):                  0.343   Jarque-Bera (JB):                2.152
Skew:                           0.338   Prob(JB):                        0.341
Kurtosis:                       2.754   Cond. No.                         12.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [342]:
# helmert contrasts on group
ols_fit2 = statsmodels.formula.api.ols("y ~ C(group, group_helmert)", data = data1).fit()

### Results Summary

In [343]:
ols_fit2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     3.832
Date:                Sat, 11 Jan 2020   Prob (F-statistic):             0.0122
Time:                        13:20:21   Log-Likelihood:                -154.23
No. Observations:                 100   AIC:                             316.5
Df Residuals:                      96   BIC:                             326.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                        2.4089      0.115     20.863      0.000       2.180       2.638
C(group, group_helmert)[H.A]     0.5443      0.163      3.334      0.001       0.220       0.868
C(group, group_helmert)[H.C]    -0.0079      0.094     -0.084      0.934      -0.195       0.179
C(group, group_helmert)[H.B]    -0.0408      0.067     -0.613      0.542      -0.173       0.091
==============================================================================
Omnibus:                        3.211   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.201   Jarque-Bera (JB):                2.944
Skew:                           0.420   Prob(JB):                        0.230
Kurtosis:                       2.991   Cond. No.                         2.45
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [344]:
# factorial contrasts (2 main effects and 1 interaction)
ols_fit3 = statsmodels.formula.api.ols("y ~ C(group, group_factorial)", data = data1).fit()

## Results Summary

In [345]:
ols_fit3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     3.832
Date:                Sat, 11 Jan 2020   Prob (F-statistic):             0.0122
Time:                        13:20:21   Log-Likelihood:                -154.23
No. Observations:                 100   AIC:                             316.5
Df Residuals:                      96   BIC:                             326.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  2.4089      0.115     20.863      0.000       2.180       2.638
C(group, group_factorial)Main Effect 1    -0.0487      0.115     -0.422      0.674      -0.278       0.180
C(group, group_factorial)Main Effect 2     0.2353      0.115      2.038      0.044       0.006       0.464
C(group, group_factorial)Interaction      -0.3091      0.115     -2.677      0.009      -0.538      -0.080
==============================================================================
Omnibus:                        3.211   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.201   Jarque-Bera (JB):                2.944
Skew:                           0.420   Prob(JB):                        0.230
Kurtosis:                       2.991   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Fit linear regression

In [346]:
ols_fit4 = statsmodels.formula.api.ols("y ~ main1 + main2 + interaction", data = data1).fit()

## Results Summary

In [347]:
ols_fit4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.107
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     3.832
Date:                Sat, 11 Jan 2020   Prob (F-statistic):             0.0122
Time:                        13:20:21   Log-Likelihood:                -154.23
No. Observations:                 100   AIC:                             316.5
Df Residuals:                      96   BIC:                             326.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       2.4089      0.115     20.863      0.000       2.180       2.638
main1          -0.0487      0.115     -0.422      0.674      -0.278       0.180
main2           0.2353      0.115      2.038      0.044       0.006       0.464
interaction    -0.3091      0.115     -2.677      0.009      -0.538      -0.080
==============================================================================
Omnibus:                        3.211   Durbin-Watson:                   1.949
Prob(Omnibus):                  0.201   Jarque-Bera (JB):                2.944
Skew:                           0.420   Prob(JB):                        0.230
Kurtosis:                       2.991   Cond. No.                         1.00
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [536]:
numpy.array([-2, -1, 1, 2]).

0.0